# Landscape or Cityscape?

The idea behind this notebook is to test out various CNNs to determine which networks are best suited to determining if an picture is an image of a city or an image of nature. My data sources for this experiment are the subreddits r/earthporn and r/cityporn. I chose these subreddits as they are focused and fairly strictly moderated. Each contains only aestetically pleasing pictures of landscapes and cityscapes respectively. I took all the directly linked images from these subreddits since 1/1/17 that have over a certain threshold of upvotes, which gave me a few thousand images that are almost entirely either a picture of a natural landscape or a picture of a cityscape

I will use keras' implementation of VGG16, ResNet50 and InceptionV3 to extract features from the images, and then I will contruct a CNN of my own design and compare the results

In [2]:
#general imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
%matplotlib inline

In [3]:
#keras specific imports
from keras import Model, Input
from keras.layers import Dense, Flatten,GlobalMaxPool2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from keras.applications import VGG16,vgg16
from keras.applications import ResNet50, resnet50
from keras.applications import InceptionV3, inception_v3

In [4]:
from project_utils import get_test

In [6]:
VGG = VGG16(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

RN50 = ResNet50(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

IV3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

87916544/87910968 [==============================] - 5s 0us/step


In [25]:
# This function takes in headless model (some pretrained CNN) and adds a top to it
def create_model(base_layers):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = GlobalMaxPool2D()(X)
    X = Dense(100, activation = 'relu',name = '1')(X)
    predictions = Dense(2 , activation = 'softmax',name = '2')(X)
    print(predictions.shape)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model
    

In [26]:
data_gen = ImageDataGenerator(shear_range= 0.2,
                                    zoom_range= 0.2,
                                    horizontal_flip= True,
                                    width_shift_range=0.2,
                                    height_shift_range = 0.2,
                                    preprocessing_function= (lambda x: x/127.5 - 1))                                           

train_generator = data_gen.flow_from_directory('./images/train', target_size= (224,224), class_mode= 'categorical')
validation_generator = data_gen.flow_from_directory('./images/validation', target_size= (224,224), class_mode= 'categorical')

vgg_model = create_model(VGG)

vgg_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',metrics = ['accuracy'])

Found 5584 images belonging to 2 classes.
Found 202 images belonging to 2 classes.
(?, 2)


In [27]:
vgg_model.fit_generator(train_generator, epochs=8, steps_per_epoch= 10,
                        validation_data=validation_generator,validation_steps=5)

Epoch 1/8
 7/10 [====================>.........] - ETA: 1:25 - loss: 1.0038 - acc: 0.6339

/home/connor/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (93153600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


10/10 [==============================] - 402s 40s/step - loss: 0.8861 - acc: 0.6750 - val_loss: 0.6774 - val_acc: 0.6125
Epoch 2/8
10/10 [==============================] - 386s 39s/step - loss: 0.3086 - acc: 0.8875 - val_loss: 0.3603 - val_acc: 0.8438
Epoch 3/8
 6/10 [=================>............] - ETA: 1:42 - loss: 0.3187 - acc: 0.8542

/home/connor/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (98130452 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


10/10 [==============================] - 399s 40s/step - loss: 0.3010 - acc: 0.8688 - val_loss: 0.5244 - val_acc: 0.7937
Epoch 4/8
10/10 [==============================] - 391s 39s/step - loss: 0.1885 - acc: 0.9250 - val_loss: 0.7084 - val_acc: 0.6937
Epoch 5/8


StopIteration: cannot identify image file './images/train/earthporn_3_7/image4551.jpg'